<h1 align='center'> Simulation of a plasma wakefield accelerator (PWFA)</h1>
<center>
Stephen D. Webb and David Bruhwiler <br>
RadiaSoft LLC <br>
swebb@radiasoft.net and bruhwiler@radiasoft.net</center>

Developed for a project supported by the United States Department of Energy, Office of Science, Office of High Energy Physics under contract number DE-SC0018718.

***
## Introduction

This notebook models a beam-driven plasma wakefield accelerator (PWFA) using nominal FACET-II parameters.

Different particle species are used for the plasma, the drive beam and the witness bunch.
***

In [1]:
## Imports

# standard python libraries
import numpy as np
from scipy import constants
from scipy.special import erfc, k0, k1

import shutil, os
import h5py as hdf5

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl

# Imports for the simulations, and setting up the plots
from fbpic.main import Simulation
from fbpic.openpmd_diag import FieldDiagnostic, ParticleDiagnostic, \
     ParticleChargeDensityDiagnostic,\
     set_periodic_checkpoint, restart_from_checkpoint
from fbpic.lpa_utils.bunch import add_elec_bunch_gaussian

class MidpointNormalize(colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

***
## Simulation Parameters

The simulation uses a moving window, beginning with the drive bunch outside the plasma, then passing through, with the simulation running until the fields reach an approximate steady state. We then use those fields to compute the wake fields and impedance.

The plasma has a density $n_e$, with the local plasma frequency given by $\omega_p = \sqrt{\frac{4 \pi n_e e^2}{m_e}}$ for the electron charge $e$ and mass $m_e$. The plasma wavenumber is $k_p = \omega_p / c$. Because PIC algorithms do not like hard edges in charge density, we will use a linear ramp on the density of the plasma, with a slope $k_{ramp}$ specified below.

In [2]:
## Domain physical parameters
print("Some physical parameters...\n")

## Plasma channel parameters
n_plasma = 4.e16  # cm^-3

# convert to per cubic meter
n_plasma *= 100**3
print "n_plasma = ", n_plasma, " [m^-3]"

# derived plasma quantities
omega_p = np.sqrt(n_plasma*constants.elementary_charge**2/(constants.m_e*constants.epsilon_0))
k_p = omega_p/constants.c

lambda_p = 2.*np.pi/k_p
print "lambda_p = ", lambda_p*1.e6, " [microns]"

## Beam parameters
# Drive bunch is gaussian
# drive_sigma_r = 3.65e-6  # meters
drive_sigma_r = 0.4 / k_p  # meters
print "R_drive_rms = ", drive_sigma_r*1.e6, " [microns]"

# drive_sigma_z = 12.77e-6  # meters
drive_sigma_z = 2. / k_p
print "Z_drive_rms = ", drive_sigma_z*1.e6, " [microns]"

# drive_Q = 1.e10*(-1.*constants.elementary_charge)   # Coulombs
drive_Q = 3.e-9   # Coulombs
print "drive_Q = ", drive_Q*1.e9, " [nC]"

# Calculate peak beam density
drive_volume_rms = 2.*np.pi*drive_sigma_z*drive_sigma_r**2
drive_dens_rms = drive_Q/np.abs(constants.elementary_charge)/drive_volume_rms
print "n_beam_peak_rms = ", drive_dens_rms, " [m^-3]"
print "n_beam/n_plasma = ", drive_dens_rms/n_plasma

drive_N_macro = 4000
drive_gamma = 1.957e+4

# Witness bunch, also gaussian
witness_sigma_r = 2.e-6 #meters
witness_sigma_z = 6.e-6  # meters
witness_Q = 1.e-10*(-1.*constants.elementary_charge)   # Coulombs
witness_N_macro = 400
witness_gamma = 100

trailing_distance = 150.e-6 # meters

## Domain parameters

# Domain size, include the whole thing and some trailing distance
domain_length = 2.*lambda_p  # meters
domain_radius = lambda_p  # meters

# Grid size, resolve the drive bunch
Delta_r = min(0.244*drive_sigma_r, 0.2*lambda_p)  # meters
Delta_z = min(Delta_r, min(0.05*drive_sigma_z, 0.1*lambda_p))  # meters
print " "
print "Delta_z = ", Delta_z*1e6, "[microns]"
print "Delta_r = ", Delta_r*1e6, "[microns]"

# Derived quantities
Nz = int(np.rint(domain_length/Delta_z))
Nr = int(np.rint(domain_radius/Delta_r))
print "Nz = ", Nz
print "Nr = ", Nr

dt = (np.sqrt((Delta_z**2 + Delta_r**2))/constants.c)  # sec
        
# Moving window
window_v = constants.c

# start the ramp after the drive bunch has existed a while

ramp_start = domain_length
ramp_length = 5.*drive_sigma_z


# create the density function for the plasma, which is uniform
def dens_func( z, r ) :
    """Returns relative density at position z and r"""
    # Allocate relative density
    n = np.ones_like(z)
    # Make linear ramp
    n = np.where( z < ramp_start + ramp_length, (z-ramp_start)/ramp_length, n )
    # Supress density before the ramp
    n = np.where( z < domain_length + ramp_start, 0., n )
    return(n)

# We want to run the simulation just long enough for the fields to form behind the drive bunch
sim_time = 2.5*domain_length/constants.c

dump_period = ( int(sim_time/dt) + 8 ) / 8
Nsteps = 8*dump_period+1
print "Nsteps = ", Nsteps

# Simplest case -- cylindrical symmetry
Nm = 1

Some physical parameters...

n_plasma =  4e+22  [m^-3]
lambda_p =  166.9471636350277  [microns]
R_drive_rms =  10.62818653107447  [microns]
Z_drive_rms =  53.14093265537234  [microns]
drive_Q =  3.0  [nC]
n_beam_peak_rms =  4.964591142034598e+23  [m^-3]
n_beam/n_plasma =  12.411477855086495
 
Delta_z =  2.5932775135821706 [microns]
Delta_r =  2.5932775135821706 [microns]
Nz =  129
Nr =  64
Nsteps =  233


***
## The Simulation

The FBPIC simulation is started in the cell below.
***

In [9]:
# remove old data
if os.path.exists('./diags/hdf5'):
    shutil.rmtree('./diags/hdf5')

# Create the simulation
sim = Simulation(Nz, domain_length, Nr, domain_radius, Nm, dt, boundaries='open', particle_shape='linear', verbose_level=2)

# By default the simulation initializes an electron species (sim.ptcl[0])
# Because we did not pass the arguments `n`, `p_nz`, `p_nr`, `p_nz`,
# this electron species does not contain any macroparticles.
# It is okay to just remove it from the list of species.
sim.ptcl = []

# plasma electrons
e_plasma = sim.add_new_species(q = -1.*constants.elementary_charge,
                               m = constants.electron_mass,
                               dens_func = dens_func, 
                               n = n_plasma, p_nz=2, p_nr=2, p_nt = 4*Nm)

# add the Gaussian drive beam
add_elec_bunch_gaussian(sim, 
                        sig_r = drive_sigma_r, 
                        sig_z = drive_sigma_z, 
                        n_emit=0., 
                        gamma0=drive_gamma, 
                        sig_gamma=1.,
                        Q=drive_Q, 
                        N=drive_N_macro, 
                        tf=0.0, 
                        zf=.75*domain_length, boost=None)

    
add_elec_bunch_gaussian(sim, 
                        sig_r = witness_sigma_r, 
                        sig_z = witness_sigma_z, 
                        n_emit=0., 
                        gamma0=drive_gamma, 
                        sig_gamma=1.,
                        Q=witness_Q, 
                        N=witness_N_macro, 
                        tf=0.0, 
                        zf=.75*domain_length-trailing_distance, boost=None)

# Set the moving window
sim.set_moving_window(v = window_v)

# Add diagnostics
sim.diags = [
            FieldDiagnostic(dump_period, sim.fld, comm=sim.comm),
    
            ParticleDiagnostic(dump_period,
                        {"plasma": e_plasma, "beam": sim.ptcl[1], "witness": sim.ptcl[2]},
                        comm=sim.comm),
    
            # The rho from `FieldDiagnostic` is total charge density.
            # It can be useful to see the charge densit of each species separately.
            ParticleChargeDensityDiagnostic(dump_period, sim,
                        {"plasma": e_plasma, "beam": sim.ptcl[1], "witness": sim.ptcl[2]})
]

# run the simulation
sim.step(Nsteps)


FBPIC (0.10.1)

MPI available: Yes
MPI processes used: 1
MPI Library Information: 
Open MPI v2.1.1, package: Open MPI mockbuild@buildhw-10.phx2.fedoraproject.org Distribution, ident: 2.1.1, repo rev: v2.1.0-100-ga2fdb5b, May 10, 2017 
CUDA available: No
Compute architecture: CPU
CPU multi-threading enabled: Yes
Threads: 40
FFT library: pyFFTW

PSATD stencil order: infinite
Particle shape: linear
Longitudinal boundaries: open
Transverse boundaries: reflective
Guard region size: 64 cells
Damping region size: 64 cells
Injection region size: 32 cells
Particle exchange period: every 10 step
Boosted frame: False

Calculating initial space charge field...
Done.

Calculating initial space charge field...
Done.

|███████████████████████████████████| 233/233, 0:00:00 left, 601 ms/step
Total time taken (with compilation): 0:02:19
Average time per iteration (with compilation): 600 ms



In [10]:
# Plot the particle and field data

file = hdf5.File('./diags/hdf5/data00000232.h5','r')
data = file.get('data/')
step = data.get('232')
ptcls = step.get('particles')
electrons = ptcls.get('plasma')
pos = electrons.get('position')

fields = step.get('fields')
rho = fields.get('rho')

# convert to number density
therho = rho[0,:,:]/constants.elementary_charge
# convert to cm^-3
therho /= 100.**3

Es = fields.get('E')
Ez = Es.get('z')
Er = Es.get('r')
theEz = Ez[0,:,:]
theEr = Er[0,:,:]

Bs = fields.get('B')
Bt = Bs.get('t')
theBt = Bt[0,:,:]

x = pos.get('x')
y = pos.get('y')
z = pos.get('z')

xPos = x[:]
yPos = y[:]
z = z[:]
r = np.sqrt(xPos**2 + yPos**2)

file.close()

In [11]:
Rads = np.linspace(0., domain_radius, Nr)
zeta = np.linspace(0., domain_length, Nz)

# move zeta so zero is centered on the drive bunch
z_avg = np.average(z)
z_avg -= constants.c * dt*(Nsteps-1)

zeta -= z_avg

zz, RR = np.meshgrid(zeta, Rads)

fig = plt.figure()

plt.imshow(therho,extent=[k_p*zeta[0],k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], cmap='viridis', origin='lower')
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$r \quad [\mu m]$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$n_e \quad [cm^{-3}]$')

plt.tight_layout()

plt.savefig('rho.png')


FigureCanvasNbAgg()

In [12]:
fig = plt.figure()

# Fix the midpoint to zero field

ezmax = np.amax(theEz*1.e-9)
ezmin = np.amin(theEz*1.e-9)
ezavg = 0.

plt.imshow(theEz*1.e-9,extent=[k_p*zeta[0], k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], 
           cmap='RdBu', origin='lower', norm=MidpointNormalize(midpoint=ezavg,vmin=ezmin, vmax=ezmax))
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$k_p r$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$E_z \quad [GV/m]$')

plt.savefig('Ez.png')


FigureCanvasNbAgg()

In [13]:
fig = plt.figure()

Ez_lineout = theEz[1,:]*1.e-9

plt.plot(k_p* zeta, Ez_lineout)
plt.xlabel(r'$k_p \zeta$')
plt.ylabel(r'$E_z$ [GV/m]')


FigureCanvasNbAgg()

Text(0,0.5,'$E_z$ [GV/m]')

In [14]:
fig = plt.figure()

Fr = (theEr - constants.speed_of_light*(theBt))*1.e-9

frmax = np.amax(Fr)
frmin = np.amin(Fr)
fravg = 0.

plt.imshow(Fr,extent=[k_p*zeta[0], k_p*zeta[-1], k_p*Rads[0], k_p*Rads[-1]], 
           cmap='RdBu', origin='lower', norm=MidpointNormalize(midpoint=fravg,vmin=frmin, vmax=frmax))
plt.xlabel(r'$\zeta = z - c t \quad [\mu m]$')
plt.ylabel(r'$r \quad [\mu m]$')
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label(r'$E_r - c B_\theta \quad [GV/m]$')
plt.title(r'Transverse force $E_r - c B_\theta$')
plt.tight_layout()

plt.savefig('Fr.png')

FigureCanvasNbAgg()

***
## References

> 1. C. B. Schroeder, D. H. Whittum, and J. S. Wurtele, "Multimode Analysis of the Hollow Plasma Channel Wakefield Accelerator", _Phys. Rev. Lett._ __82__, 1177 (1999). [https://doi.org/10.1103/PhysRevLett.82.1177](https://doi.org/10.1103/PhysRevLett.82.1177)

> 2. R. Lehe, M. Kirchen, I. A. Andriyash, B. B. Godfrey, and J.-L. Vay, "A spectral, quasi-cylindrical and dispersion-free Particle-In-Cell algorithm", _Comp. Phys. Comm._ __203__, pp. 66-82 (2016). [https://doi.org/10.1016/j.cpc.2016.02.007](https://doi.org/10.1016/j.cpc.2016.02.007)

> 3. C. Joshi _et al._ "Plasma wakefield acceleration experiments at FACET II", _Plasma Phys. Control. Fusion_ __60__, 3 (2018).

> 4. A. W. Chao, "Physics of Collective Beam Instabilities in High Energy Accelerators", John Wiley & Sons (1993)

> 5. C. A. Lindstrom _et al._ "Measurement of Transverse Wakefields Induced by a Misaligned Positron Bunch in a Hollow Channel Plasma Accelerator", _Phys. Rev. Lett._ __120__, 124802 (2018).